In [ ]:
base_experiment_path = "data\\plays"
experiment_path = f"{base_experiment_path}\\play"
device = "cpu"

In [ ]:
!{sys.executable} -m pip install huggingface_sb3
!{sys.executable} -m pip install stable_baselines3
!{sys.executable} -m pip install gym
!{sys.executable} -m pip install shimmy 

## Load Model / Policy

In [ ]:
#from huggingface_sb3 import load_from_hub
#
#checkpoint = load_from_hub(
#	repo_id="sb3/dqn-MountainCar-v0",
#	filename="dqn-MountainCar-v0.zip",
#)
#
#from stable_baselines3 import DQN
#
#model_sb3 = DQN.load(checkpoint)

## Setup Model / Policy

In [ ]:
from automl.external_support.sb3.sb3_policy_wrappers import SB3Wrapper


policy = (SB3Wrapper, {"sb3_model" : "dqn-MountainCar-v0"})

## Setup Environment

In [ ]:
#from automl.rl.environment.gymnasium_env import GymnasiumEnvironmentWrapper
#
# #env_class = GymnasiumEnvironmentWrapper
# 
env_input = {"environment" : "MountainCar-v0", "render_mode" : "human", "device" : device}

In [ ]:
from automl.rl.environment.gymnasium_env import GymnasiumEnvironmentWrapperSampler

env_class = GymnasiumEnvironmentWrapperSampler

env_input = {"environment_input" : env_input}

In [ ]:
env = (env_class, env_input)


## Setup Single Agent

In [ ]:
from automl.rl.agent.agent_components import AgentSchema

policy_input = {}


agent_input = {
    "policy": policy,

}

agent = (AgentSchema, 
         agent_input)

## Setup all Agents

In [ ]:
all_agents_input = {
    "device" : device,
}

In [ ]:
agents = agent

## Setup player

In [ ]:
from automl.rl.rl_player.rl_player import RLPlayer

rl_player_input = {
    "base_directory" : experiment_path,
    "artifact_relative_directory" : "play",
    "create_new_directory" : True,
    "agents" : agents,
    "agents_input" : all_agents_input,
    "num_episodes" : 5,
    "store_env_at_end" : True    
    }

rl_player_class = RLPlayer

rl_player = (rl_player_class, rl_player_input)

In [ ]:
from automl.rl.evaluators.rl_evaluator_player import EvaluatorWithPlayer

evaluator_with_player = EvaluatorWithPlayer({
    
            "rl_player_definition" : rl_player,
            "number_of_episodes" : 3, # number of episodes per environment
            "number_of_evaluations" : 3, # number of sampled environments essentially
            "environment" : env

})


## Play

In [ ]:
#rl_player.run()
from automl.utils.files_utils import open_or_create_folder

experiment_path = open_or_create_folder(experiment_path, create_new=True)

print(f"Using experiment path {experiment_path} from base experiment path {experiment_path}")

evaluator_with_player.evaluate((agents, device, experiment_path, env))

## Store results

In [ ]:
from automl.utils.json_component_utils import save_configuration


save_configuration(evaluator_with_player, experiment_path, "config.json", save_exposed_values=True, ignore_defaults=False)